# Tokyo Housing Database & Price Forecasting

In [ ]:
#Install the 'ipython-sql' and 'prettytable' libraries using pip
!pip install ipython-sql prettytable

# Import necessary Python modules for API calls, JSON handling, SQLite, datetime, and data analysis
import requests, json, sqlite3, sys, re
import pandas as pd
import numpy as np
import prettytable 
from bs4 import BeautifulSoup
prettytable.DEFAULT = 'DEFAULT'

# Load SQL magic extension to run SQL queries directly in notebook cells
%load_ext sql

In [10]:
# Path to the SQLite database where scraped housing data will be stored
db = 'tokyo_housing.db'

# Base URL of Suumo (Japanese housing site)
base_url = 'https://suumo.jp/'

# URLs for initial listings pages 
starting_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=25.0&md=01&md=02&md=03&md=04&md=05&md=06&md=07&md=08&md=09&md=10&et=20&mb=0&mt=9999999&cn=9999999&ra=013&ek=035017990&ek=035026830&rn=0350&ae=03501'

In [11]:
# Define TokyoHousingScraper to:
# - Scrape Tokyo housing listings from Suumo.jp
# - Collect listings, parse property details
# - Store the results in SQLite database

class TokyoHousingScraper:
    
    def __init__(self, db, base_url, url):
        # Initialize DB connection
        self.db = db
        self.conn = sqlite3.connect(self.db)
        self.cursor = self.conn.cursor()

        # Base URL and starting page
        self.base_url = base_url
        self.url = url
    
    def scrape_listings(self):
		# Define list for storing HTML
        self.listings = list()
		
		# Iterate through all pages of listings
        while True:
            try:
                response = requests.get(next_page) #this will only work after the first page
            except:
                response = requests.get(self.url) # starting url 
            soup = BeautifulSoup(response.text, 'lxml')
			
			# Each listing = cassetteitem div
            cassettes = soup.select('div.cassetteitem')
            self.listings.extend(cassettes)
            print(len(self.listings))

			# Find next page link (pagination)
            try:
                current_page = soup.find('li', class_ = 'pagination-current')
                next_page_path = current_page.find_next_siblings('li')[1]
                next_page = self.base_url + next_page_path.select_one('a').get('href')
            except: break #no more pages to comb through

        print(f'{len(self.listings)} listings were successfully gathered!')

    def parse_station_info(self, item):
		
        # Extract station information (names, distances, nearest, average).
        # Returns tuple: 
            # (stations_str, nearest_station, distance_to_nearest_station, avg_distance).

		# Get raw station blocks
        stations_list = item.select('li.cassetteitem_detail-col2 div.cassetteitem_detail-text'
		) if item.select(
			'li.cassetteitem_detail-col2 div.cassetteitem_detail-text'
		) else None

		# If there is no station information, return None
        if stations_list == None:
            return (None, None, None, None)
        else: pass

		# Remove empty tags
        stations_list = [s for s in stations_list if s != '']

		# All stations as a single string (for DB storage)
        self.stations_str = ",".join([station.get_text().strip() for station in stations_list])
	
		# Extract stations and distances with regex
        stations_dict = {
			# All listed stations
			'stations': [
				re.findall(r'/(?P<station>.*?)\s*歩', station.get_text().strip())[0] 
				for station in stations_list
				if re.findall(r'/(?P<station>.*?)\s*歩', station.get_text().strip())
			],
		
			'distances': [
				re.findall(r'\d+', station.get_text().strip())[0]
				for station in stations_list
				if re.findall(r'\d+', station.get_text().strip())
			]
		}
	
		# Compute distance to nearest station
        self.distance_to_nearest_station = min([int(dist) for dist in stations_dict['distances']])
        nearest_idx = stations_dict['distances'].index(str(self.distance_to_nearest_station))
        self.nearest_station = stations_dict['stations'][nearest_idx]
	
		# Compute average distance to surrounding stations
        self.avg_distance = np.mean([float(dist) for dist in stations_dict['distances']])

        return self.stations_str, self.nearest_station, self.distance_to_nearest_station, self.avg_distance

    def build_housing_dataset(self):
		
		# Extract housing data (title, rent, floor, area, stations, etc.)
            # and save into SQLite as a DataFrame.

        self.housing_data = [
		{
			'img': item.select_one(
                'div.cassetteitem_object img'
            ).get('rel') if item.select_one(
                'div.cassetteitem_object img'
            ) else None,
            
            'title': item.select_one(
				'div.cassetteitem_content-title'
			).get_text().strip() if item.select_one(
				'div.cassetteitem_content-title'
			) else None,
		
			'address': item.select_one(
				'li.cassetteitem_detail-col1'
			).get_text().strip() if item.select_one(
				'li.cassetteitem_detail-col1'
			) else None,
		
			'rent': item.select_one(
				'span.cassetteitem_price.cassetteitem_price--rent'
			).get_text().strip() if item.select_one(
				'div.cassetteitem-item span.cassetteitem_price.cassetteitem_price--rent'
			) else None,
		
			'management_fee': item.select_one(
				'span.cassetteitem_price.cassetteitem_price--administration'
			).get_text().strip() if item.select_one(
				'span.cassetteitem_price.cassetteitem_price--administration'
			) else None,
		
			'deposit': item.select_one(
				'span.cassetteitem_price.cassetteitem_price--deposit'
			).get_text().strip() if item.select_one(
				'span.cassetteitem_price.cassetteitem_price--deposit'
			) else None,

			'key_money': item.select_one(
				'span.cassetteitem_price.cassetteitem_price--gratuity'
			).get_text().strip() if item.select_one(
				'span.cassetteitem_price.cassetteitem_price--gratuity'
			) else None,

			'floor': item.select(
				'div.cassetteitem-item tr.js-cassette_link td'
			)[2].get_text().strip() if item.select(
				'div.cassetteitem-item tr.js-cassette_link td'
			)[2] else None,
		
			'floor_plan': item.select_one(
				'span.cassetteitem_madori'
			).get_text().strip() if item.select_one(
				'span.cassetteitem_madori'
			) else None,

			'area': item.select_one(
				'span.cassetteitem_menseki'
			).get_text().strip() if item.select_one(
				'span.cassetteitem_menseki'
			) else None,

			'building_age': item.select(
				'li.cassetteitem_detail-col3 div'
			)[0].get_text().strip() if item.select(
				'li.cassetteitem_detail-col3'
			) else None,

			'building_size': item.select(
				'li.cassetteitem_detail-col3 div'
			)[1].get_text().strip() if item.select(
				'li.cassetteitem_detail-col3'
			) else None,

			'stations': self.parse_station_info(item)[0],

			'nearest_station': self.parse_station_info(item)[1],

			'distance_to_nearest_station': self.parse_station_info(item)[2],

			'avg_distance_to_stations': self.parse_station_info(item)[3]
		}
			for item in self.listings
		]

		# Save to DataFrame + SQLite
        self.housing_data_df = pd.DataFrame(self.housing_data)
        self.housing_data_df.to_sql(name = 'HOUSING_DATA', con = self.conn, if_exists = 'replace', index = False)

		# Close the DB connection
        self.conn.close()

In [12]:
# Instantiate scraper
scraper = TokyoHousingScraper(db, base_url, starting_url)

# Scrape housing listings
scraper.scrape_listings()

# Parse listing details and save dataset to SQLite
scraper.build_housing_dataset()

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1211
1211 listings were successfully gathered!


In [18]:
# Connect to SQLite database for querying listings 
conn = sqlite3.connect(db)
cursor = conn.cursor()

# Initialize SQL Magic with database connection
%sql sqlite:///tokyo_housing.db

In [ ]:
%%sql 
-- Remove the view if it already exists
DROP VIEW IF EXISTS TOKYO_HOUSING;

-- Create a cleaned + feature-engineered housing view
CREATE VIEW TOKYO_HOUSING AS

WITH STANDARDIZED_LISTINGS AS (
    SELECT 
        -- Basic identifiers
        img, title, address, 
        
        -- Convert rent/deposit/key money into numeric
        CAST(RTRIM(rent, '万円') AS FLOAT) * 10000 AS rent,
        CAST(RTRIM(management_fee, '円') AS INTEGER) AS management_fee,
        CAST(RTRIM(deposit, '万円') AS FLOAT) * 10000 AS deposit,
        CAST(RTRIM(key_money, '万円') AS FLOAT) * 10000 AS key_money,
        
        -- Convert floor to integer
        CAST(RTRIM(floor, '階') AS INTEGER) AS floor,
        
        -- Normalize floor plan labels 
        CASE
            WHEN floor_plan = 'ワンルーム' THEN '1R'
            ELSE floor_plan
        END AS floor_plan,
        
        -- Convert area to numeric (square meters)
        CAST(RTRIM(area, 'm2') AS FLOAT) AS area,
        
        -- Extract building age in years
        CAST(LTRIM(RTRIM(building_age, '年'), '築') AS INTEGER) AS building_age,
        
        -- Standardize building size
        CASE
            WHEN building_size LIKE '地下%' THEN 
                CAST(SUBSTR(building_size, 3, 1) AS INTEGER) +
                CAST(SUBSTR(building_size, 6, 1) AS INTEGER)
            WHEN building_size LIKE '地上%' THEN
                CAST(SUBSTR(building_size, 3, 1) AS INTEGER)
            ELSE CAST(RTRIM(building_size, '階建') AS INTEGER)
        END AS building_size,
        
        -- Station-related features
        stations,
        nearest_station,
        distance_to_nearest_station,
        ROUND(avg_distance_to_stations, 2) AS avg_distance_to_stations
    FROM HOUSING_DATA
),

FEATURED_LISTINGS AS (
    SELECT 
        img, title, address, rent, 
        
        -- Replace 0/invalid values with NULLs
        NULLIF(management_fee, 0) AS management_fee,
        NULLIF(deposit, -0.0) AS deposit,
        NULLIF(key_money, 0.0) AS key_money,
        floor, floor_plan, area, building_age,
        building_size, nearest_station,
        distance_to_nearest_station, avg_distance_to_stations,
        
        -- Feature engineering: average rents by station, floor plan, and distance to nearest station
        ROUND(AVG(rent) 
            OVER (PARTITION BY nearest_station), 2) 
            AS avg_rent_by_station, 
        ROUND(AVG(rent)
            OVER (PARTITION BY floor_plan), 2) 
            AS avg_rent_by_floor_plan,
        
        -- Price rank relative to other listings near the same station
        DENSE_RANK() 
            OVER (PARTITION BY nearest_station ORDER BY rent DESC)
            AS price_rank_by_station
    FROM STANDARDIZED_LISTINGS
)

-- Final output 
SELECT * FROM FEATURED_LISTINGS

In [ ]:
# Query the engineered SQL view into a pandas DataFrame for analysis
tokyo_housing = %sql SELECT * FROM TOKYO_HOUSING 
tokyo_housing_df = tokyo_housing.DataFrame()

# Close the DB connection 
conn.close()

In [16]:
# Drop duplicate listings 
tokyo_housing_df.drop_duplicates(inplace = True)

In [17]:
# Save DataFrame to CSV file
tokyo_housing_df.to_csv('tokyo_housing.csv', index = False)